In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions  import from_unixtime
from pyspark.sql.functions  import to_date
from pyspark.sql import Row
from pyspark.sql.functions import to_json, struct
from pyspark.sql import functions as F

# Mount storage

Databricks use blob storage (ADLS Gen-2) as data source, here's an example of how to mount ADLS Gen-2 Storage FileSystem to DBFS.

First, create a ADLS-Gen2 storage account, and define mount point.

In [ ]:
storageAccount="xxx" # Name of the ADLS Gen2 Storage Account
mountpoint = "/mnt/xxx" # Mount the storage account to a chosen path in DBFS
storageEndPoint ="abfss://container_name@{}.dfs.core.windows.net/".format(storageAccount)

Then, authenticate into the storage endpoint. To do so, these needs to be performed in Azure:
1. **Application registration**: You will need to register an Azure Active Directory (AAD) application. On the Azure portal home page, search for "Azure Active Directory" &rarr; select App registrations &rarr; New registration.
2. **Create secret to the application**: Click on "Certificates & secrets" under the Manage heading &rarr; add a new client secret &rarr; Copy the value
3. **Grant ADLS-Gen2 access to the registered Application**: In the ADLS-Gen2 storage account, navigate to Access Control (IAM) &rarr; Add &rarr; Add role assignment &rarr; Role = Storage Blob Data Contributor; Assign access to = User, group, or service principal; Select = The registered Application

In [ ]:
clientID ="xxx" # Obtained from (1) the registered Application -> Application (client) ID
tenantID ="xxx" # Obtained from (1) the registered Application -> Directory (tenant) ID
clientSecret ="xxx" # Obtained from (2) the registered Application -> Copied secret value
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)


configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientID,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}

try:
  dbutils.fs.mount(
  source = storageEndPoint,
  mount_point = mountpoint,
  extra_configs = configs)
except Exception as e:
  if 'Directory already mounted' in str(e):
    print('Directory already mounted')
  else:
    print(str(e))


These are some useful commands to inspect the mounts

In [ ]:
# List all mount points
display(dbutils.fs.mounts())

# List files under a specific mount point
display(dbutils.fs.ls(mountpoint))

Files in the ADLS-Gen2 can thus be read

In [ ]:
# Reading Orders.csv file in a Spark dataframe
df_ord= spark.read.format("csv").option("header",True).load("dbfs:/mnt/Gen2/csvFiles")

In [ ]:
# Executing the below will create an External table in Databricks for you to read the CSV.
# This is not about creating Delta tables however
spark.sql(f"""
CREATE OR REPLACE TABLE Orders
    {{(schema)}}
USING {{csv}}
OPTIONS (
    path 'dbfs:/mnt/Gen2/csvFiles',
    header 'true',
    delimiter ','
    )
""")

# Reading and writing to Delta Tables 

Delta Table is actually a bunch of snappy-compressed parquet files, with Delta Log files. It offer the following benefits:
- Easy rollback since it tracks every changes to the table in the delta log
- ACID compliance
- Enforce constraint defined in DDL
- Optimized performance

Define paths

In [ ]:
# Mount path for the raw data blobs
original_path = "/mnt/Gen2/Orders/someFiles"

# Delta output path
delta_path = "/mnt/Gen2/Orders/delta"

Create Delta table

This is an example for creating Delta Table using CSV files, these will happen after executing the below:
- Under the defined LOCATION, snappy-compressed parquet files will be created, alongside the _delta_log folder
- You will find the relevant table under the "Data" section in Databricks

The table information will persist in Databricks's metastore even when you shut down the clusters

In [ ]:
# The Temp View is necessary to allow Databricks to parse the CSV first, before making it a Delta table
# LOCATION flag is used to make the table an unmanaged one, whose data does not reside in Databricks but in the specified path
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW Orders_vw
    {{(schema, if necessary)}}
USING {{file format e.g. csv}}
OPTIONS (
    path '{original_path}',
    header 'true',
    delimiter '|'
    );

CREATE OR REPLACE TABLE Orders
LOCATION '{delta_path}'
PARITION BY {{column name}}
AS
SELECT *, EXTRACT(YEAR FROM Event_Date) FROM Orders_vw
""")

For self-describing file formats like parquet, the syntax can be made easier

In [ ]:
# Note: This will not work for CSV, beecause the CTAS statement cannot infer schema correctly
spark.sql(f"""
CREATE OR REPLACE TABLE Orders
LOCATION '{delta_path}'
PARITION BY {{column name}}
AS
    SELECT *
    FROM parquet.`{original_path}`
"""
)

Alternative code using PySpark

In [ ]:
# Read input into dataframe
df_ord = (spark.read.format("parquet").load(original_path)
      .withColumn("timestamp", current_timestamp())
      .withColumn("O_OrderDateYear", year(col("O_OrderDate")))
     )

# Save into delta path. Go over to the ADLS Gen2 container and you should see new files got created in the delta path
# Files are organized into different folders according to the "partitionBy" value
df_ord.write.format("delta").partitionBy("O_OrderDateYear").mode("overwrite").save(delta_path)

# Execute this then visit the "Data" section in Databricks, you will see the relevant table.
spark.sql(f"""
CREATE OR REPLACE TABLE Orders
LOCATION '{delta_path}'
""")

After creation, the table can be queried in SQL or Python

In [ ]:
# Via SQL
%sql
SELECT o.*
FROM Orders o

# Via Python
deltaTable = spark.read.format("delta").load(delta_path)
deltaTable.show()

Some useful commands

In [ ]:
%sql

# View history of the Delta table
DESCRIBE HISTORY Orders

# Query a specific version
SELECT * FROM Orders VERSION AS OF 1

# Restore a previous version
RESTORE TABLE Orders VERSION AS OF 5

# View details of the Delta table e.g. number of files, partitioning, etc.
DESCRIBE DETAIL Orders

Tables vs Views vs CTE
- Table
    - Managed table: Data is actually stored in DBFS
    - Unmanaged table: Data is stored in elsewhere e.g. ADLS-Gen2
- Views
    - View: Will persist like table
    - Temp View: Persist in the current notebook session only
    - Global Temp View: Can be shared across different notebook sessions, until cluster restarts
- CTE
    - CTE: Referenced within the scope of a SQL statement only


# Streaming data pipeline from EventHub

This is an example of building the Bronze, Silver, and Gold Zone for a streaming data pipeline
- Bronze: Read live data from EventHub, and historical data from ADLS-Gen2. Parse content and union them
- Silver: Implement business rules and data cleansing process and join with lookup tables
- Gold: Data is aggregated

In [ ]:
# Define variables
db_name = "VehicleSensor"

def get_config(zone):
    return {
    'delta_path': f"/mnt/SensorData/vehiclestreamingdata/{zone}/delta",
    'chkpt_path': f"/mnt/SensorData/vehiclestreamingdata/{zone}/chkpt",
    'delta_table': f"VehicleDelta_{zone}"
    }

# Create DB first
spark.sql(f"CREATE DATABASE IF NOT EXISTS{db_name}")

## Bronze Zone

### Streaming from Kafka

Create Spark DataFrame which reads from the Kafka topic

In [ ]:
TOPIC = "cookbook-eventhub" # Event Hub namespace
BOOTSTRAP_SERVERS = "cookbook-eventhub.servicebus.windows.net:9093" # Host name of Event Hub:9093, 9093 is the port for Kafka

# Go to Event Hub's Shared Access Policies -> Click onto a policy -> Copy Connection string–primary key here
CONN_STRING = "Endpoint=sb://kafkaenabledeventhubns.servicebus.windows.net/;SharedAccessKeyName=sendreceivekafka;SharedAccessKey=4vxbVwasdasdsdasd4aVcUWBvYp44sdasaasasasasasasvoVE=" 

# The $ConnectionString and $Default are fixed values, don't update them
EH_SASL = EH_SASL = f"kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"{CONN_STRING}\";"
GROUP_ID = "$Default" 

# // Read stream using Spark SQL (structured streaming)
# // consider adding .option("startingOffsets", "earliest") to read earliest available offset during testing
kafkaDF = spark.readStream \
    .format("kafka") \
    .option("subscribe", TOPIC) \
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.jaas.config", EH_SASL) \
    .option("kafka.request.timeout.ms", "60000") \
    .option("kafka.session.timeout.ms", "60000") \
    .option("kafka.group.id", "POC") \
    .option("failOnDataLoss", "false") \
    .option("startingOffsets", "latest") \
    .load() \
    .withColumn("source", lit(TOPIC)) # Optional: Also add the topic as column

#Check if streaming is on and getting the schema for the kakfa dataframe 
print(kafkaDF.isStreaming)
print(kafkaDF.printSchema())

#It should then output something like this:
#
#True
#root
# |-- key: binary (nullable = true)
# |-- value: binary (nullable = true)
# |-- topic: string (nullable = true)
# |-- partition: integer (nullable = true)
# |-- offset: long (nullable = true)
# |-- timestamp: timestamp (nullable = true)
# |-- timestampType: integer (nullable = true)
# |-- source: string (nullable = true)

Parse the Kafka message and writing the streaming data to Delta table

In [ ]:
# Creating the schema for the 'value' field in Kafka message
jsonschema = StructType() \
      .add("id", StringType()) \
      .add("eventtime", TimestampType()) \
      .add("rpm", IntegerType()) \
      .add("speed", IntegerType()) \
      .add("kms", IntegerType()) \
      .add("lfi", IntegerType())  \
      .add("lat", DoubleType()) \
      .add("long", DoubleType())

def parse_to_bronze(kafkaDF, value_schema):

      # Parse, and write.
      # Checkpoint is set so that it can recover from failure in the event of server failure
      # Trigger is for controlling the frequency of writes. For incremental batch jobs. availableNow=True is recommended
      parsedDF=kafkaDF.selectExpr("CAST(key AS STRING) as key", "CAST(value AS STRING) as value", "source") \
            .withColumn('vehiclejson', from_json(col('value'), value_schema)) \
            .select("key", "value", "source", "vehiclejson.*") \
            .writeStream.format("delta") \
            .trigger(processingTime = "2 minutes") \
            .outputMode("append") \
            .option("checkpointLocation",get_config("Bronze")['chkpt_path']) \
            .start(get_config("Bronze")['delta_path']) 

      return parsedDF

# Create writeStream
parse_to_bronze(kafkaDF, jsonschema)
# Data can thus be streamed to a Delta table 
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {db_name}.{get_config("Bronze")['delta_table']}
USING DELTA LOCATION '{get_config("Bronze")['delta_path']}'
""")

### Streaming from ADLS-Gen2

In case you have historical data in ADLS-Gen2, and you would like to stream-read the data as files are ingested there, AutoLoader is the prefered method.

This method is also useful for batch jobs, as likely should be the case here

In [ ]:
# Function to read data from ADLS gen-2 using readStream API and writing as delta format
def append_batch_source(data_source_mnt_path, value_schema):
  
  kafkaDF = (spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", get_config("Hist")['chkpt_path'])
    .load(data_source_mnt_path)
  )

  parsedDF=kafkaDF.withColumn("source", lit('historical')) \
            .selectExpr("CAST(key AS STRING) as key", "CAST(value AS STRING) as value", "source") \
            .withColumn('vehiclejson', from_json(col('value'), value_schema)) \
            .select("key", "value", "source", "vehiclejson.*") \
            .writeStream.format("delta") \
            .trigger(availableNow=True) \
            .option("checkpointLocation",get_config("Hist")['chkpt_path']) \
            .option("mergeSchema", "true") \
            .outputMode("append") \
            .start(get_config("Hist")['delta_path']) 

  return parsedDF

# Create historical delta table
append_batch_source('/xxx', jsonschema)
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {db_name}.{get_config("Historical")['delta_table']}
USING DELTA LOCATION '{get_config("Historical")['delta_path']}'
""")

In [ ]:
# Inspect schema
spark.sql(f"""
DESCRIBE FORMATTED {db_name}.{get_config("Bronze")['delta_table']}
""")

### Union the live and historical data, and generate Temp View

With live and historical delta tables created, we can union them for subsequent usage

In [ ]:
#Streaming Data from Bronze and Historical tables
df_bronze=spark.readStream.format("delta").option("latestFirst", "true").table(f"{db_name}.{get_config("Bronze")['delta_table']}")
df_historical=spark.readStream.format("delta").option("latestFirst", "true").table(f"{db_name}.{get_config("Historical")['delta_table']}")

#Joining both historical and Bronze Streaming Data. The TempView can be used like CTE in SQL statements
df_bronze_all = df_bronze.union(df_historical)
df_bronze_all.createOrReplaceTempView("vw_TempBronzeAll")

In [ ]:
%sql
select count(*) from vw_TempBronzeAll

## Silver Zone

### Connect to Azure SQL DB for lookup tables

Establish connection to Azure SQL DB

In [ ]:
# Config details for Azure SQL DB for VehicleInformation and LocationInformation tables
sqldbusername = dbutils.secrets.get(scope="KeyVaultScope",key="VehicleInformationDBUserId")
sqldbpwd = dbutils.secrets.get(scope="KeyVaultScope",key="VehicleInformationDBPwd")

jdbcHostname = "vehicledemoinformatiosrvr.database.windows.net"
jdbcDatabase = "VehicleInformationDB"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4}".format(jdbcHostname, jdbcPort, jdbcDatabase, sqldbusername, sqldbpwd)
connectionProperties = {
  "user" : sqldbusername,
  "password" : sqldbpwd,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

Retrieve the two lookup tables and create Temp Views

In [ ]:
# Reading dbo.VehicleInfo master table from Azure SQL DB and creating a view
vehicleInfo = "(select VehicleId,Make,Model,Category,ModelYear from dbo.VehicleInformation) vehicle"
df_vehicleInfo = spark.read.jdbc(url=jdbcUrl, table=vehicleInfo, properties=connectionProperties)
df_vehicleInfo.createOrReplaceTempView("vw_VehicleMaster")
display(df_vehicleInfo)

In [ ]:
# Reading dbo.LocationInfo master table from Azure SQL DB and creating a view
locationInfo = "(select Borough,Location,Latitude,Longitude from dbo.LocationInfo) vehicle"
df_locationInfo = spark.read.jdbc(url=jdbcUrl, table=locationInfo, properties=connectionProperties)
df_locationInfo.createOrReplaceTempView("vw_LocationMaster")
display(df_locationInfo)

### De-duplication

Because Kafka provides at-least-once guarantees on data delivery, all Kafka consumers should be prepared to handle duplicate reocrds

In [ ]:
deduped_df = (spark.readStream
                   .table("vw_TempBronzeAll")
                   .select("*")
                   .withWatermark("time", "30 seconds")
                   .dropDuplicates(["id", "eventtime"])
                   .createOrReplaceTempView("vw_TempBronzeAll_dedup"))

Delta Lake has optimized functionality for insert-only merges. This operation is ideal for de-duplication: define logic to match on unique keys, and only insert those records for keys that don't already exist.

Note that in this application, we proceed in this fashion because we know two records with the same matching keys represent the same information. If the later arriving records indicated a necessary change to an existing record, we would need to change our logic to include a **`WHEN MATCHED`** clause.

A merge into query is defined in SQL below against a view titled **`stream_updates`**.

In [ ]:
sql_query = f"""
  MERGE INTO {get_config("Silver")['delta_table']} a
  USING (SELECT
    stream_updates.id
    ,stream_updates.eventtime
    ,stream_updates.rpm
    ,Year(eventtime) as Year
    ,month(eventtime) as Month
    ,day(eventtime) as Day
    ,hour(eventtime) as Hour
    ,m.Make
    ,m.Model
    ,m.Category
    ,l.Borough
    ,l.Location
    FROM stream_updates
    LEFT JOIN vw_VehicleMaster m on stream_updates.id = m.VehicleId
    LEFT join vw_LocationMaster l on stream_updates.lat = l.Latitude and stream_updates.long = l.Longitude
    ) b
  ON a.VehicleId=b.VehicleId AND a.eventtime=b.eventtime
  WHEN NOT MATCHED THEN INSERT *
"""

The Spark Structured Streaming **`foreachBatch`** method allows users to define custom logic when writing.

The logic applied during **`foreachBatch`** addresses the present microbatch as if it were a batch (rather than streaming) data, thus enabling the use of some functions, like merge, or window functions. Otherwise it would return error, see the end of this section

The class defined in the following cell defines simple logic that will allow us to register any SQL **`MERGE INTO`** query for use in a Structured Streaming write.

In [ ]:
def upsert_to_delta(microBatchDF, batchId):
    microBatchDF.createOrReplaceTempView("stream_updates")
    microBatchDF._jdf.sparkSession().sql("""
    MERGE INTO {get_config("Silver")['delta_table']} a
    USING (SELECT
        stream_updates.id
        ,stream_updates.eventtime
        ,stream_updates.rpm
        ,Year(eventtime) as Year
        ,month(eventtime) as Month
        ,day(eventtime) as Day
        ,hour(eventtime) as Hour
        ,m.Make
        ,m.Model
        ,m.Category
        ,l.Borough
        ,l.Location
        FROM stream_updates
        LEFT JOIN vw_VehicleMaster m on stream_updates.id = m.VehicleId
        LEFT join vw_LocationMaster l on stream_updates.lat = l.Latitude and stream_updates.long = l.Longitude
        ) b
    ON a.VehicleId=b.VehicleId AND a.eventtime=b.eventtime
    WHEN NOT MATCHED THEN INSERT *
    """)

Because we're using SQL to write to our Delta table, we'll need to make sure this table exists before we begin.

In [ ]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {get_config("Silver")['delta_table']}
(id STRING, eventtime TIMESTAMP, rpm INTEGER, Year INTEGER, Month INTEGER, Day INTEGER, Hour INTEGER, 
Make STRING, Model STRING, Category STRING, Borough STRING, Location STRING)
USING DELTA
LOCATION {get_config("Silver")['delta_path']}
""")

### Perform the upsert

Use the previously defined function in our **`foreachBatch`** logic.

In [ ]:
df_silver = (deduped_df.writeStream
            .format("delta") \
            .foreachBatch(upsert_to_delta)
            .outputMode("update") \
            .option("checkpointLocation",get_config("Silver")['chkpt_path'])  \
            .start()
)

Run the following code to read data as streaming data from the Delta table.

In [ ]:
display(spark.readStream.format("delta").table(get_config("Silver")['delta_table']).groupBy("Make").count().orderBy("Make"))

Note: If we try to apply this to a streaming read of our data without using ```foreachBatch```, we'll learn that
> Non-time-based windows are not supported on streaming DataFrames

Below is an example for an attempt to perform a window-function based deduplication

In [ ]:
# ranked_df = (spark.readStream
#                   .table("bronze")
#                   .filter("topic = 'user_info'")
#                   .select(F.from_json(F.col("value").cast("string"), schema).alias("v"))
#                   .select("v.*")
#                   .filter(F.col("update_type").isin(["new", "update"]))
#                   .withColumn("rank", F.rank().over(window))
#                   .filter("rank == 1").drop("rank"))

# display(ranked_df)

### Quality enforcement 

This can be implemented by:
1. Enforcing Table Constraints, and sending the unconforming data to a Delta table
2. Adding a CASE WHEN flag to the Silver table itself

Below is a demonstration of #1

Table constraints apply boolean filters to columns within a table and prevent data that does not fulfill these constraints from being written.

In [ ]:
ALTER TABLE heart_rate_silver ADD CONSTRAINT validbpm CHECK (heartrate > 0);

Creating a table to store quarantined records

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS bpm_quarantine
    (device_id LONG, time TIMESTAMP, heartrate DOUBLE)
USING DELTA
LOCATION '${da.paths.user_db}/bpm_quarantine'

With Structured Streaming operations, writing to an additional table can be accomplished within **`foreachBatch`** logic.

In [ ]:
sql_query = """
MERGE INTO heart_rate_silver a
USING stream_updates b
ON a.device_id=b.device_id AND a.time=b.time
WHEN NOT MATCHED THEN INSERT *
"""

class Upsert:
    def __init__(self, query, update_temp="stream_updates"):
        self.query = query
        self.update_temp = update_temp 
        
    def upsert_to_delta(self, micro_batch_df, batch):
        micro_batch_df.filter("heartrate" > 0).createOrReplaceTempView(self.update_temp)
        micro_batch_df._jdf.sparkSession().sql(self.query)
        micro_batch_df.filter("heartrate" <= 0).write.format("delta").mode("append").saveAsTable("bpm_quarantine")

Note that within the **`foreachBatch`** logic, the DataFrame operations are treating the data in each batch as if it's static rather than streaming.

As such, we use the **`write`** syntax instead of **`writeStream`**.

This also means that our exactly-once guarantees are relaxed. In our example above, we have two ACID transactions:
1. Our SQL query executes to run an insert-only merge to avoid writing duplicate records to our silver table.
2. We write a microbatch of records with negative heartrates to the **`bpm_quarantine`** table

If our job fails after our first transaction completes but before the second completes, we will re-execute the full microbatch logic on job restart.

However, because our insert-only merge already prevents duplicate records from being saved to our table, this will not result in any data corruption.

## Gold Zone

Aggregate the data and write to Gold Delta table

In [ ]:
# Save data in Delta format
df_gold=(
spark.readStream.format("delta").option("latestFirst", "true").table(f"{db_name}.{get_config("Silver")['delta_table']}")
    # Apply Watermark to handle late data and perform aggeregation. See here for a discussion about withWatermark and outputMode
    # https://dvirgiln.github.io/spark-structured-streaming-output-modes/
    .withWatermark("timestamp","4 minutes")
    .groupBy(window('eventtime',"1 hour"),"Make","Borough","Location","Month","Day","Hour").count()) \
        .writeStream.format("delta") \
        .outputMode("update") \
        .option("checkpointLocation", get_config("Gold")['chkpt_path']) \
        .start(get_config("Gold")['delta_path'])

# Create Delta table
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {db_name}.{get_config("Gold")['delta_table']}
USING DELTA LOCATION '{get_config("Gold")['delta_path']}'
""")

# Delta Live Tables

Delta Live Tables allows SQL statements to be structured as ELT pipelines. SQLs can be written as usual, but with a few differences:
- Adding the LIVE word preceding the TABLE
- References to DLT tables and views will always include the ```live.``` prefix. Which allows this to be automatically substituted at runtime, providing convenience for pipeline migration across DEV/QA/Prod environments

Here's an example for using DLT + Auto Loader for Bronze -> Silver -> Gold incremental processing

## Bronze Zone

Incremental processing via <a herf="https://docs.databricks.com/spark/latest/structured-streaming/auto-loader.html" target="_blank">Auto Loader</a> (which uses the same processing model as Structured Streaming), requires the addition of the **`STREAMING`** keyword in the declaration as seen below. The **`cloud_files()`** method enables Auto Loader to be used natively with SQL. This method takes the following positional parameters:
* The source location, as mentioned above
* The source data format, which is JSON in this case
* An arbitrarily sized array of optional reader options. In this case, we set **`cloudFiles.inferColumnTypes`** to **`true`**

In [ ]:
CREATE OR REFRESH STREAMING LIVE TABLE sales_orders_raw
COMMENT "The raw sales orders, ingested from retail-org/sales_orders."
AS SELECT * FROM cloud_files("_mnt_path_sales_orders_", "json", map("cloudFiles.inferColumnTypes", "true"));

CREATE OR REFRESH STREAMING LIVE TABLE customers
COMMENT "The customers buying finished products, ingested from retail-org/customers."
AS SELECT * FROM cloud_files("_mnt_path_customers_", "csv");

## Silver Zone

The **`CONSTRAINT`** keyword introduces quality control. Similar in function to a traditional **`WHERE`** clause, **`CONSTRAINT`** integrates with DLT, enabling it to collect metrics on constraint violations. Constraints provide an optional **`ON VIOLATION`** clause, specifying an action to take on records that violate the constraint. The three modes currently supported by DLT include:

| **`ON VIOLATION`** | Behavior |
| --- | --- |
| **`FAIL UPDATE`** | Pipeline failure when constraint is violated |
| **`DROP ROW`** | Discard records that violate constraints |
| Omitted | Records violating constraints will be included (but violations will be reported in metrics) |

In [ ]:
CREATE OR REFRESH STREAMING LIVE TABLE sales_orders_cleaned(
  CONSTRAINT valid_order_number EXPECT (order_number IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "The cleaned sales orders with valid order_number(s)."
AS
  SELECT f.customer_id, f.customer_name, f.number_of_line_items, 
         timestamp(from_unixtime((cast(f.order_datetime as long)))) as order_datetime, 
         date(from_unixtime((cast(f.order_datetime as long)))) as order_date, 
         f.order_number, f.ordered_products, c.state, c.city, c.lon, c.lat, c.units_purchased, c.loyalty_segment
  FROM STREAM(LIVE.sales_orders_raw) f
  LEFT JOIN LIVE.customers c
    ON c.customer_id = f.customer_id
    AND c.customer_name = f.customer_name

## Gold Zone

At the most refined level of the architecture, we declare a table delivering an aggregation with business value, in this case a collection of sales order data based in a specific region. In aggregating, the report generates counts and totals of orders by date and customer.

In [ ]:
CREATE OR REFRESH LIVE TABLE sales_order_in_la
COMMENT "Sales orders in LA."
AS
  SELECT city, order_date, customer_id, customer_name, ordered_products_explode.curr, 
         sum(ordered_products_explode.price) as sales, 
         sum(ordered_products_explode.qty) as quantity, 
         count(ordered_products_explode.id) as product_count
  FROM (SELECT city, order_date, customer_id, customer_name, explode(ordered_products) as ordered_products_explode
        FROM LIVE.sales_orders_cleaned 
        WHERE city = 'Los Angeles')
  GROUP BY order_date, city, customer_id, customer_name, ordered_products_explode.curr

# Delta Change Data Feed

Change data feed allows Databricks to track row-level changes between versions of a Delta table. When enabled on a Delta table, the runtime records change events for all the data written into the table.

Here are the codes for enabling this feature:

In [ ]:
# This enables CDF for particular tables
spark.sql("""
ALTER TABLE silverTable SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
"""
)

# Or ,enable CDF using Spark conf setting in a notebook or on a cluster will ensure it's used on all newly created Delta tables in that scope.
spark.conf.set("spark.databricks.delta.properties.defaults.enableChangeDataFeed", True)

After this is enabled, two things will happen:
- A folder ```_change_data``` will appear in the Delta Table directory, which contains parquet files
- CDC data of the table can be read using the below methods

In [ ]:
# Via Python
cdc_df = (spark.readStream
               .format("delta")
               .option("readChangeData", True)
               .option("startingVersion", 0)
               .table("silver"))

# Via SQL
spark.sql("""
SELECT * FROM table_changes('silver', 0) order by _commit_timestamp
""")

Note that we are using a table that has updates written to it as a streaming source! This is a **huge** value add, and something that historically has required extensive workarounds to process correctly.

This would mean:
- **Silver and Gold tables**: Updates to Silver table can be isolated and pushed to Gold table, without reading through the whole Silver table. Data removal requests can also be more easily fulfilled, as DELETE pushed down, see below cell for an example
- **Materialized view**s: Create up-to-date, aggregated views of information for use in BI and analytics without having to reprocess the full underlying tables, instead updating only where changes have come through.
- **Transmit changes**: Send a change data feed to downstream systems such as Kafka or RDBMS that can use it to incrementally process in later stages of data pipelines.
- **Audit trail table**: Capture the change data feed as a Delta table provides perpetual storage and efficient query capability to see all changes over time, including when deletes occur and what updates were made.

Please see [here](https://docs.databricks.com/_static/notebooks/delta/cdf-demo.html) for a demo

In [ ]:
def process_deletes(microBatchDF, batchId):
    
    (microBatchDF
        .filter("_change_type = 'delete'")
        .createOrReplaceTempView("deletes"))
    
    microBatchDF._jdf.sparkSession().sql("""
        MERGE INTO users u
        USING deletes d
        ON u.alt_id = d.alt_id
        WHEN MATCHED
            THEN DELETE
    """)

    microBatchDF._jdf.sparkSession().sql("""
        DELETE FROM user_bins
        WHERE user_id IN (SELECT user_id FROM deletes)
    """)
    
    microBatchDF._jdf.sparkSession().sql("""
        MERGE INTO delete_requests dr
        USING deletes d
        ON d.alt_id = dr.alt_id
        WHEN MATCHED
          THEN UPDATE SET status = "deleted"
    """)

# Other things

## Adding Commit Messages

Delta Lake supports arbitrary commit messages that will be recorded to the Delta transaction log and viewable in the table history. This can help with later auditing.

Setting this with SQL will create a global commit message that will be used for all subsequent operations in our notebook.

In [ ]:
SET spark.databricks.delta.commitInfo.userMetadata=Deletes committed

With DataFrames, commit messages can also be specified as part of the write options using the **`userMetadata`** option.

Here, we'll indicate that we're manually processing these requests in a notebook, rather than using an automated job.

In [ ]:
query = (requests_df.writeStream
                    .outputMode("append")
                    .option("checkpointLocation", f"{DA.paths.checkpoints}/delete_requests")
                    .option("userMetadata", "Requests processed interactively")
                    .trigger(availableNow=True)
                    .table("delete_requests"))

## Archiving Data
If a company wishes to maintain an archive of historic records (but only maintain recent records in production tables), cloud-native settings for auto-archiving data can be configured to move data files automatically to lower-cost storage locations.

The cell below simulates this process (here using copy instead of move). 

Note that because only the data files and partition directories are being relocated, the resultant table will be Parquet by default.

**NOTE**: For best performance, directories should have **`OPTIMIZE`** run to condense small files to 1GB each. Because valid and stale data files are stored side-by-side in Delta Lake files, partitions should also have **`VACUUM`** executed prior to moving any Delta Lake data to a pure Parquet table to ensure only valid files are copied.

In [ ]:
archive_path = f"{DA.paths.working_dir}/pii_archive"
source_path = f"{DA.paths.user_db}/bronze/topic=user_info"

files = dbutils.fs.ls(source_path)
[dbutils.fs.cp(f[0], f"{archive_path}/{f[1]}", True) for f in files if f[1][-8:-1] <= '2019-48'];

spark.sql(f"""
CREATE TABLE IF NOT EXISTS user_info_archived
USING parquet
LOCATION '{archive_path}'
""")

spark.sql("MSCK REPAIR TABLE user_info_archived")

display(spark.sql("SELECT COUNT(*) FROM user_info_archived"))

Here we'll model deleting all **`user_info`** that was received before week 49 of 2019.

Note that we are deleting cleanly along partition boundaries. All the data contained in the specified **`week_part`** directories will be removed from our table.

In [ ]:
DELETE FROM bronze 
WHERE topic = 'user_info'
AND week_part <= '2019-48';

VACUUM bronze RETAIN 0 HOURS

However, with the deletes, the pipelines would need to be updated as well as there should now be exceptions
> Detected deleted data from streaming source

To enable streaming processing from Delta tables with partition deletes, add the **`.option("ignoreDeletes", True)`** to the DataStreamReader. 

## Optimization
- Remove unused files from a table directory via [VACCUM](https://learn.microsoft.com/en-us/azure/databricks/spark/latest/spark-sql/language-manual/delta-vacuum/), add DRY RUN to preview previous versions to be deleted first. This is especially important for deleting PII information, as after DELETE the PII might still exist in previous images of the data
- Compacting small files and collocate related information in the same set of files via [OPTIMIZE and ZORDER](https://www.confessionsofadataguy.com/exploring-delta-lakes-zorder-and-performance-on-databricks/).
- Turning on Auto Optimize and Auto Compaction help us avoid the tables containing too many small files. For more information on these settings, see [here](https://docs.databricks.com/delta/optimizations/auto-optimize.html)


# Orcchestration and Scheduling

Parameters can be added to tasks

In [ ]:
# Use the widgets API to retreive job parameters
value = dbutils.widgets.get("name")
# Inject into context for use in the following SQL statements
spark.conf.set("name", value) 

In [ ]:
INSERT INTO task_4 VALUES ('From ${name}', current_timestamp())